# Predicción de Altas de Tarjetas de Crédito
### Prueba Técnica - Selección de Variables Significativas y Optimización del Umbral

En este notebook se desarrolla un modelo de Machine Learning para predecir si un cliente adquirirá una tarjeta de crédito (`alta_tdc`).
El proceso se divide en dos etapas principales:

1. **Selección de Variables Significativas:** Se entrena un modelo base para extraer la importancia de cada variable y, posteriormente, se optimiza el umbral de selección para elegir las variables más relevantes.
2. **Construcción y Evaluación del Modelo Final:** Se construye un pipeline final que incluye preprocesamiento, balanceo de clases (SMOTE) y entrenamiento de un RandomForestClassifier con hiperparámetros predeterminados.

El objetivo es obtener un modelo robusto y, al final, mostrar únicamente el average_precision_score y la precisión.

## Objetivo y Entregables

**Objetivo:** Desarrollar un modelo predictivo utilizando únicamente las variables significativas, optimizando el umbral de selección y entrenando un modelo con parámetros predeterminados, de manera que se impriman únicamente el average_precision_score y la precisión.

**Entregables:**
- **Notebook (.ipynb):** Con el desarrollo del modelo, incluyendo la justificación de las decisiones tomadas y una reflexión sobre posibles mejoras o próximos pasos.
- **Archivo de predicciones (.csv):** Con las columnas `ID_cliente` y `alta_tdc` (1 si el cliente adquiere la tarjeta, 0 en caso contrario).

In [28]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, average_precision_score,f1_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [29]:
# Cargar el dataset de entrenamiento
train_df = pd.read_csv('train_df.csv', delimiter=';', thousands='.', decimal=',')
print('Dataset de entrenamiento cargado, dimensiones:', train_df.shape)
train_df.head()

Dataset de entrenamiento cargado, dimensiones: (12012, 165)


C:\Users\wilson.hernandez\AppData\Local\Temp\ipykernel_22992\3922022781.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('train_df.csv', delimiter=';', thousands='.', decimal=',')


,identificador_cliente,cust_age_number,customer_seniority_number,main_city_type,town_id,personal_type,marital_status_type,address_relationship_type,adult_dependants_number,target_type,...,cr_dbt_card_tech_expns_amount,cr_debit_card_tech_op_number,cr_dbt_cd_tourism_expns_amount,cr_debit_cd_tourism_op_number,cr_dbt_cd_clothes_expns_amount,cr_debit_cd_clothes_op_number,cr_dbt_card_prvs_expns_amount,cr_debit_card_prvs_op_number,audit_date,alta_tdc
0,1,68,37,1,11001,1,1.0,3,0.0,1,...,12545000.0,3.0,0.0,0.0,8500000.0,1.0,18852200.0,5.0,2024-11-07T05:18:36.416Z,0
1,2,92,37,1,11001,1,2.0,3,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-07T05:18:36.416Z,0
2,3,70,27,0,25572,1,2.0,2,NaN,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-11-07T05:18:36.416Z,0
3,4,76,27,0,25290,1,2.0,3,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-07T05:18:36.416Z,0
4,5,90,37,1,5001,1,2.0,5,2.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-11-07T05:18:36.416Z,0


In [30]:
# Definir variables objetivo e identificación
target = 'alta_tdc'
id_col = 'identificador_cliente'

# Separar características y target
X = train_df.drop([target, id_col], axis=1)
y = train_df[target]

# Identificar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=['number']).columns.tolist()
cat_cols = X.select_dtypes(exclude=['number']).columns.tolist()

print('Columnas numéricas:', num_cols)
print('Columnas categóricas:', cat_cols)

# Convertir variables categóricas a string
X[cat_cols] = X[cat_cols].astype(str)

Columnas numéricas: ['cust_age_number', 'customer_seniority_number', 'main_city_type', 'marital_status_type', 'address_relationship_type', 'adult_dependants_number', 'target_type', 'customer_asset_amount', 'customer_liability_amount', 'customer_status_type', 'month_withdrawals_number', 'month_payment_number', 'month_purchase_number', 'm_exit_amount', 'm_entry_amount', 'mth_mbl_app_qry_number', 'mth_mbl_app_mnty_tr_number', 'mth_mbl_app_non_mnty_tr_number', 'month_atm_queries_number', 'atm_made_fin_trans_number', 'atm_made_nfin_tran_number', 'inquiries_digi_channel_number', 'dig_financial_trans_tot_number', 'dig_non_fin_trans_tot_number', 'mth_co_netcash_queries_number', 'mth_co_netc_non_mnty_tr_number', 'mth_co_netcash_mnty_tr_number', 'mth_ppl_netcash_queries_number', 'mth_ppl_netcash_mnty_tr_number', 'mth_pplnetc_non_mnty_tr_number', 'month_h2h_queries_number', 'month_h2h_mnty_tr_number', 'month_h2h_non_mnty_tr_number', 'month_webpage_queries_number', 'mth_webpage_mnty_tr_number', 'm

## Modelo Base y Cálculo de Importancia de Variables

Se construye un pipeline que procesa las variables numéricas y categóricas y se entrena un **RandomForestClassifier** para obtener la importancia de cada variable.

Decisiones tomadas:
- Uso de **SimpleImputer** para valores faltantes (mediana para numéricas y moda para categóricas).
- Aplicación de **StandardScaler** y **OneHotEncoder** para normalización y codificación respectivamente.

In [31]:
# Pipeline para variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combinar ambos pipelines
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

# Crear pipeline base: preprocesamiento + RandomForestClassifier
pipeline_base = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# División en entrenamiento y prueba
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline_base.fit(X_train_base, y_train_base)

# Extraer el clasificador entrenado
model_base = pipeline_base.named_steps['classifier']

# Obtener nombres de columnas tras preprocesamiento
onehot_feature_names = pipeline_base.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(cat_cols)
feature_names = num_cols + list(onehot_feature_names)

# Calcular importancias
importances = model_base.feature_importances_
feat_imp = pd.Series(importances, index=feature_names).sort_values(ascending=False)

print('Importancia de las variables:')
print(feat_imp.head(10))

Importancia de las variables:
customer_seniority_number      0.036184
cust_age_number                0.034910
customer_liability_amount      0.029448
current_payroll_incm_amount    0.028778
mth_mbl_app_qry_number         0.028758
m_entry_amount                 0.027503
saving_prdt_avg_bal_amount     0.026915
month_withdrawals_number       0.023876
month_purchase_number          0.022724
customer_asset_amount          0.022421
dtype: float64


## Optimización del Umbral de Importancia

Se explora un rango de umbrales sobre las importancias para seleccionar las variables más relevantes, reduciendo la complejidad del modelo al evitar incluir variables con baja contribución.

In [32]:
# Definir un rango de umbrales (de 0 al máximo de importancias)
umbral_values = np.linspace(0, feat_imp.max(), num=20)

best_umbral = None
best_score = 0
best_num_cols = None
best_cat_cols = None

for umbral in umbral_values:
    features_temp = feat_imp[feat_imp > umbral].index.tolist()
    
    num_cols_temp = [col for col in num_cols if col in features_temp]
    
    onehot_all_names = pipeline_base.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(cat_cols)
    cat_cols_temp = []
    for col in cat_cols:
        dummies = [dummy for dummy in onehot_all_names if dummy.startswith(col + '_')]
        if any(dummy in features_temp for dummy in dummies):
            cat_cols_temp.append(col)
    
    selected_cols_temp = num_cols_temp + cat_cols_temp
    if len(selected_cols_temp) == 0:
        continue
    
    X_selected_temp = X[selected_cols_temp]
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X_selected_temp, y, test_size=0.2, random_state=42)
    
    numeric_transformer_sel = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    categorical_transformer_sel = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    preprocessor_sel = ColumnTransformer(transformers=[
        ('num', numeric_transformer_sel, [col for col in selected_cols_temp if col in num_cols]),
        ('cat', categorical_transformer_sel, [col for col in selected_cols_temp if col in cat_cols])
    ])
    clf_temp = Pipeline(steps=[
        ('preprocessor', preprocessor_sel),
        ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
    ])
    
    clf_temp.fit(X_train_temp, y_train_temp)
    y_pred_temp = clf_temp.predict(X_test_temp)
    
    # Utilizamos el F1 Score internamente para la optimización (no se imprime)
    score = 0
    try:
        score = f1_score(y_test_temp, y_pred_temp, pos_label=1)
    except Exception as e:
        pass
    
    if score > best_score:
        best_score = score
        best_umbral = umbral
        best_num_cols = num_cols_temp
        best_cat_cols = cat_cols_temp

print('Mejor umbral:', best_umbral)
print('Variables numéricas seleccionadas:', best_num_cols)
print('Variables categóricas seleccionadas:', best_cat_cols)

Mejor umbral: 0.009522048051814731
Variables numéricas seleccionadas: ['cust_age_number', 'customer_seniority_number', 'marital_status_type', 'address_relationship_type', 'customer_asset_amount', 'customer_liability_amount', 'month_withdrawals_number', 'month_purchase_number', 'm_exit_amount', 'm_entry_amount', 'mth_mbl_app_qry_number', 'mth_mbl_app_mnty_tr_number', 'atm_made_fin_trans_number', 'mth_ppl_netcash_queries_number', 'mth_ppl_netcash_mnty_tr_number', 'month_h2h_mnty_tr_number', 'mth_ext_pos_mnty_tr_number', 'current_payroll_incm_amount', 'saving_prdt_avg_bal_amount', 'consumer_cash_balance_amount', 'bbva_payrl_oblg_tot_bal_amount', 'avg_cr_card_balance_amount', 'customer_products_number', 'cr_dbt_card_veh_expns_amount', 'cr_dbt_cd_courier_expns_amount', 'cr_dbt_card_spmk_expns_amount', 'cr_debit_card_spmk_op_number']
Variables categóricas seleccionadas: ['town_id']


## Construcción del Pipeline Final con Variables Seleccionadas

Utilizando el umbral óptimo, se seleccionan las variables relevantes y se crea el pipeline final que incluye:
- Preprocesamiento para las variables numéricas y categóricas seleccionadas.
- Balanceo de clases mediante **SMOTE**.
- Entrenamiento de un **RandomForestClassifier** con parámetros predeterminados.

Al final se mostrarán únicamente el average_precision_score y la precisión.

In [33]:
selected_cols = best_num_cols + best_cat_cols
print('Columnas seleccionadas para el modelo final:', selected_cols)

numeric_transformer_sel = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer_sel = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor_sel = ColumnTransformer(transformers=[
    ('num', numeric_transformer_sel, [col for col in selected_cols if col in num_cols]),
    ('cat', categorical_transformer_sel, [col for col in selected_cols if col in cat_cols])
])

clf_final = Pipeline(steps=[
    ('preprocessor', preprocessor_sel),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

Columnas seleccionadas para el modelo final: ['cust_age_number', 'customer_seniority_number', 'marital_status_type', 'address_relationship_type', 'customer_asset_amount', 'customer_liability_amount', 'month_withdrawals_number', 'month_purchase_number', 'm_exit_amount', 'm_entry_amount', 'mth_mbl_app_qry_number', 'mth_mbl_app_mnty_tr_number', 'atm_made_fin_trans_number', 'mth_ppl_netcash_queries_number', 'mth_ppl_netcash_mnty_tr_number', 'month_h2h_mnty_tr_number', 'mth_ext_pos_mnty_tr_number', 'current_payroll_incm_amount', 'saving_prdt_avg_bal_amount', 'consumer_cash_balance_amount', 'bbva_payrl_oblg_tot_bal_amount', 'avg_cr_card_balance_amount', 'customer_products_number', 'cr_dbt_card_veh_expns_amount', 'cr_dbt_cd_courier_expns_amount', 'cr_dbt_card_spmk_expns_amount', 'cr_debit_card_spmk_op_number', 'town_id']


In [34]:
X_selected = X[selected_cols]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

In [35]:
clf_final_smote = ImbPipeline(steps=[
    ('preprocessor', preprocessor_sel),
    ('smote', SMOTE(random_state=42)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

clf_final_smote.fit(X_train, y_train)

y_pred_smote = clf_final_smote.predict(X_test)

# Calcular average_precision_score y precisión
ap_smote = average_precision_score(y_test, clf_final_smote.predict_proba(X_test)[:, 1])
accuracy = accuracy_score(y_test, y_pred_smote)

print("Average Precision (SMOTE):", ap_smote)
print("Precisión:", accuracy)

Average Precision (SMOTE): 0.7394994286632312
Precisión: 0.9563046192259675


## Generación de Predicciones y Archivo de Salida

Se generan las predicciones sobre el conjunto de validación y se guarda el resultado en un archivo CSV con las columnas:
- **ID_cliente**
- **alta_tdc** (1 si el cliente adquiere la tarjeta, 0 en caso contrario)

In [36]:
validation_df = pd.read_csv('Validation_df.csv', delimiter=';', thousands='.', decimal=',')

X_validation = validation_df[selected_cols]

validation_preds = clf_final_smote.predict(X_validation)

output_df = pd.DataFrame({
    'ID_cliente': validation_df[id_col],
    'alta_tdc': validation_preds
})

output_df.to_csv('predicciones.csv', index=False, sep=';')
print('Archivo de predicciones generado: predicciones.csv')

Archivo de predicciones generado: predicciones.csv


## Justificación de las Decisiones y Próximos Pasos

Decisiones tomadas:

- Se entrenó un modelo base para obtener la importancia de las variables y se optimizó el umbral para seleccionar las más relevantes.
- Se utilizó un **RandomForestClassifier** con parámetros predeterminados para mantener la simplicidad.
- Se aplicó **SMOTE** para balancear las clases y mejorar la generalización del modelo.

**Reflexión y Próximos Pasos:**

- Explorar otros algoritmos (p.ej., XGBoost o LightGBM).
- Investigar técnicas adicionales de balanceo y validación cruzada.
- Documentar cada decisión para asegurar la claridad y reproducibilidad.